In [21]:
%run McienceSchema.ipynb
import numpy as np
import pandas as pd
import boto3
from fnmatch import fnmatch
import os
import re
import io
import smtplib 
from pretty_html_table import build_table
from datetime import datetime
from email.mime.text import MIMEText
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.header import Header
from pathlib import Path

In [22]:
def getS3Credentials():
    file_path = "C:/Users/abhishek.sriramoju_s/Desktop/MscienceCredentials.txt"  #File path
    try:
        with open(file_path, 'r') as file:  #Opening the file that contains credentials
            content = file.read()  #Reading the file
    except FileNotFoundError:  
        print(f"File not found at {file_path}")
        content = ""


    aws_access_key = None  
    aws_secret_key = None
    service_name = None
    region_name = None

    lines = content.split('\n') #Spliting the string on the basis of next line
    for line in lines:
        if line.startswith("service_name"):    #Getting the Service Name
            service_name = line.split('=')[1].strip()
        elif line.startswith("region_name"):   #Getting the Region Name
            region_name = line.split('=')[1].strip()
        elif line.startswith("AWS_ACCESS_KEY_ID"): #Getting the Access key
            aws_access_key = line.split('=')[1].strip()
        elif line.startswith("AWS_SECRET_ACCESS_KEY"): #Getting the Secret key
            aws_secret_key = line.split('=')[1].strip()
        else :
            bucket_name = line.split('=')[1].strip()
            
    return aws_access_key,aws_secret_key,service_name,region_name,bucket_name

In [23]:
#Connecting to amazon s3 bucket
def buildConnection(service_name,region_name,aws_access_key_id,aws_secret_access_key):
    resource = boto3.resource(service_name = service_name, #Service Name
                        region_name = region_name,   #Region
                        aws_access_key_id = aws_access_key, #Access key
                        aws_secret_access_key = aws_secret_key) #Secrect key
    return resource               

In [25]:
def getFiles(resource,bucket_name) :
    bucket_name = bucket_name  #Bucket Name
    categories_bucket = resource.Bucket(bucket_name) #Getting Bucket object
    categories_files = categories_bucket.objects.filter(Prefix='tokopedia_pe_legacy/')  #Getting the objects of category file
    category_dates = []  
    categories_files
    for obj in categories_files :  #Iterating on category files
        name = obj.key  #Category file name
        if fnmatch(name, "*tokopedia_sku_pe_legacy*") and fnmatch(name, "*IsBackfillData=1*") and fnmatch(name, "*.parquet*") :
            values = name.split('_')   #Splitting the file names
            values = values[5].split('/')[2]  #Will get the date in yyyy.mm.dd
            date = list(map(int, values.split('.'))) 
            category_dates.append((name, datetime(date[0], date[1], date[2]))) #Appending datet and file name in category_dates

    category_dates.sort(key = lambda x: x[1],reverse = True) #Sorting the list on the basis of date
    return categorydates

In [24]:
aws_access_key,aws_secret_key,service_name,region_name,bucket_name = getS3Credentials()
resource = buildConnection(service_name,region_name,aws_access_key,aws_secret_key)
latestFile,previousFile = getFiles()

s3.ServiceResource()